In [ ]:
import tweepy
from datetime import datetime,timezone
import pytz
import pandas as pd
import json
import numpy as np
import time


root = "./"
output_path = root+"output/"

In [2]:
# twitter api 情報
api_key =  "xxx"
api_key_secret = "xxx"
access_token = "xxx"
access_token_secret = "xxx"
bearer_token = "xxx"

#Twitterの認証
client = tweepy.Client(
    consumer_key=api_key,
    consumer_secret=api_key_secret,
    access_token=access_token,
    access_token_secret=access_token_secret,
    bearer_token=bearer_token, 
    return_type="dict", 
    wait_on_rate_limit=True
)

In [3]:
# 検索条件の設定
search_word = "みずほ -みずほ町 -みずほちゃん"
# 最大検索数
max_search_num = 100

In [6]:
def get_tweets_and_save(search_word, max_search_num=100, csv_name="tweet_data.csv"):
    def write_csv(df, csv_name="tweet_data.csv"):
        prev_df = pd.read_csv(output_path+csv_name)
        df = pd.concat([prev_df, df])
        df = df.reset_index(drop=True)
        df = df[~df["text"].duplicated()]
        df.to_csv(output_path+csv_name, index=False)
    
    # jsonの展開
    def add_public_metrics_column(df):
        df = df.reset_index(drop=True)
        public_metrics = np.array(df["public_metrics"])
        # public_metrics = np.array([s.replace("'", '"') for s in public_metrics])

        df["retweet_count"] = 0
        df["reply_count"] = 0
        df["like_count"] = 0
        df["quote_count"] = 0
        
        for i, d in enumerate(public_metrics):
            # d = json.loads(d)
            for key, value in d.items():
                df.at[i, key] = value

        return df.drop("public_metrics", axis=1)
    
    
    # 最新のツイートを取得
    tweets = client.search_recent_tweets(
        query=search_word, 
        max_results=max_search_num,
        tweet_fields = ['author_id', 'created_at', 'public_metrics'],
        user_fields = 'profile_image_url',
        expansions = ['author_id', 'attachments.media_keys'],
        # media_fields = 'url',
    )

    # df化
    df = pd.DataFrame(tweets[0])
    
    # jsonの展開
    df = add_public_metrics_column(df)
    
    # 保存
    write_csv(df, csv_name)

In [ ]:
while True:
    # ----- みずほ
    try:
        get_tweets_and_save("みずほ -みずほ町 -みずほちゃん", max_search_num, "tweet_data.csv")
        
    except Exception as e:
        print(e)
        print("    retry -> ", end="")
        
        # retry
        try:
            get_tweets_and_save("みずほ -みずほ町 -みずほちゃん", max_search_num, "tweet_data.csv")
            print("successfully")
            
        except Exception as e:
            print("failed")
            print(e)
    # -----
            
    
    # ----- 銀行
    try:
        get_tweets_and_save("銀行", max_search_num, "tweet_data_bank.csv")
        
    except Exception as e:
        print(e)
        print("    retry -> ", end="")
        
        # retry
        try:
            get_tweets_and_save("銀行", max_search_num, "tweet_data_bank.csv")
            print("successfully")
            
        except Exception as e:
            print("failed")
            print(e)
    # -----
    
    # 15分おき
    time.sleep(60*15)